# ใบงานที่ 11: การแปลงสัญญาณแอนะล็อกเป็นดิจิทัล (ADC) ใน ESP32

## วัตถุประสงค์
1. เพื่อทำความเข้าใจหลักการทำงานของ ADC (Analog-to-Digital Converter)
2. เพื่อศึกษาคุณสมบัติและข้อจำกัดของ ADC ใน ESP32
3. เพื่อฝึกการเขียนโปรแกรมอ่านค่าจากเซนเซอร์แอนะล็อกด้วย ESP32
4. เพื่อวิเคราะห์และแปลความหมายข้อมูลที่ได้จาก ADC

## อุปกรณ์ที่ต้องใช้
- ESP32 Development Board
- Potentiometer (ตัวต้านทานปรับค่าได้) 10kΩ 
- LDR (Light Dependent Resistor) หรือ Photoresistor 
- ตัวต้านทาน 10kΩ สำหรับ pull-down
- Breadboard และสายไฟต่อวงจร
- ESP-IDF บน VSCODE

## ภาคปฏิบัติการทดลอง
- การทดลองที่ 1 - การอ่านค่า Potentiometer
- การทดลองที่ 2 - การอ่านค่า LDR
- การทดลองที่ 3 - การปรับปรุงความแม่นยำของ ADC


------

## ภาคทฤษฎี: ความรู้พื้นฐานเกี่ยวกับ ADC

### 1. ADC คืออะไร?
ADC (Analog-to-Digital Converter) เป็นวงจรที่ทำหน้าที่แปลงสัญญาณแอนะล็อก (ต่อเนื่อง) เป็นสัญญาณดิจิทัล (ไม่ต่อเนื่อง) ที่ไมโครคอนโทรลเลอร์สามารถประมวลผลได้

### 2. กระบวนการแปลงสัญญาณ ADC
การแปลงสัญญาณ ADC ประกอบด้วย 3 ขั้นตอนหลัก:

1. **การสุ่มสัญญาณ (Sampling)**: การดึงตัวอย่างสัญญาณในช่วงเวลาที่กำหนด
2. **การจัดระดับสัญญาณ (Quantization)**: การแบ่งช่วงแรงดันเป็นระดับต่างๆ
3. **การเข้ารหัส (Encoding)**: การแปลงระดับแรงดันเป็นตัวเลขฐานสอง

### 3. คุณสมบัติสำคัญของ ADC
- **Resolution (ความละเอียด)**: จำนวนบิตที่ใช้แทนค่า เช่น 12-bit $ADC = 2^{12} = 4096$ ระดับ
- **Reference Voltage (แรงดันอ้างอิง)**: ช่วงแรงดันที่ ADC สามารถวัดได้
- **Sampling Rate**: ความเร็วในการสุ่มสัญญาณ (ตัวอย่างต่อวินาทรี)

## คุณสมบัติ ADC ของ ESP32

ESP32 มี ADC ที่มีคุณสมบัติดังนี้:

### ADC Channel และ Pin
- **ADC1**: 8 channels (GPIO32-39) 
    
    เนื่องจาก ESP32 เจาะจงใช้งานขาเหล่านี้เป็นอินพุตเท่านั้น  ควรเลือกใช้งานขาเหล่านี้สำหรับการอ่านค่าแอนะล็อกก่อนที่จะใช้ขาอื่นๆ เพื่อจะได้นำขาอื่นๆ ไปใช้เป็นดิจิทัลหรือฟังก์ชันอื่นๆ
- **ADC2**: 10 channels (GPIO0, 2, 4, 12-15, 25-27)
- **ความละเอียด**: 12-bit (0-4095)
- **แรงดันอ้างอิง**: 0-3.3V (ปรับได้ด้วย attenuation)

### Attenuation Settings
ESP32 สามารถปรับช่วงการวัดแรงดันได้ดังนี้:
- `ADC_ATTEN_DB_0`: 0-1.1V
- `ADC_ATTEN_DB_2_5`: 0-1.5V  
- `ADC_ATTEN_DB_6`: 0-2.2V
- `ADC_ATTEN_DB_11`: 0-3.9V (แนะนำสำหรับ 3.3V system)

### ข้อจำกัดของ ADC ใน ESP32
1. ADC2 ไม่สามารถใช้งานได้เมื่อเปิด WiFi
2. ความไม่เป็นเชิงเส้น (Non-linearity) โดยเฉพาะที่แรงดันสูงและต่ำ
3. Noise และ interference จาก WiFi/Bluetooth

## การคำนวณแปลงค่า ADC เป็นแรงดัน

### สูตรการแปลงค่า

$$V_{out} = \frac{ADC_{value}}{ADC_{resolution}} \times V_{ref}$$

โดยที่:
- $V_{out}$ = แรงดันเอาต์พุต (V)
- $ADC_{value}$ = ค่าดิจิทัลที่อ่านได้จาก ADC
- $ADC_{resolution}$ = ความละเอียดของ ADC (สำหรับ 12-bit = $2^{12} - 1 = 4095$)
- $V_{ref}$ = แรงดันอ้างอิง (Reference Voltage)

### ตัวอย่างการคำนวณ
สำหรับ ESP32 ที่ใช้ 12-bit ADC และ attenuation 11dB:
- $ADC_{resolution} = 4095$ (2¹² - 1)
- $V_{ref} = 3.3V$
- หาก $ADC_{value} = 2048$

แทนค่าในสมการ:

$$V_{out} = \frac{2048}{4095} \times 3.3V = 0.5003 \times 3.3V \approx 1.65V$$

### ความละเอียดของการวัด

ความละเอียดต่ำสุดที่วัดได้ (LSB - Least Significant Bit):

$$LSB = \frac{V_{ref}}{2^n}$$

โดยที่ $n$ = จำนวนบิตของ ADC

สำหรับ ESP32 (12-bit):

$$LSB = \frac{3.3V}{2^{12}} = \frac{3.3V}{4096} \approx 0.806mV$$

## ภาคปฏิบัติ: การทดลองที่ 1 - การอ่านค่า Potentiometer

### วัตถุประสงค์
เพื่อศึกษาการทำงานของ ADC ด้วยการอ่านค่าจาก Potentiometer

### วงจร
เชื่อมต่อ Potentiometer กับ ESP32 ดังนี้


    "![ESP32 Potentiometer Wiring](https://raw.githubusercontent.com/koson/Week-11-Microcontroller-applications/main/Images/ESP32-R_pot.png)\n",

- ขา 1 ของ Potentiometer ต่อกับ GND
- ขา 2 ของ Potentiometer ต่อกับ GPIO34
- ขา 3 ของ Potentiometer ต่อกับ 3.3V

### โค้ดตัวอย่าง

In [ ]:
// การทดลองที่ 1: อ่านค่า Potentiometer
// ตัวอย่างพื้นฐาน ADC ของ ESP32 โดยใช้ ESP-IDF

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "freertos/FreeRTOS.h"
#include "freertos/task.h"
#include "driver/adc.h"
#include "esp_adc_cal.h"
#include "esp_log.h"

// กำหนด pin ที่ใช้
#define POTENTIOMETER_CHANNEL ADC1_CHANNEL_6  // GPIO34 (ADC1_CH6)
#define DEFAULT_VREF    1100        // ใช้ adc2_vref_to_gpio() เพื่อให้ได้ค่าประมาณที่ดีกว่า
#define NO_OF_SAMPLES   64          // การสุ่มสัญญาณหลายครั้ง

static const char *TAG = "ADC_POT";
static esp_adc_cal_characteristics_t *adc_chars;

static bool check_efuse(void)
{
    // ตรวจสอบว่า TP ถูกเขียนลงใน eFuse หรือไม่
    if (esp_adc_cal_check_efuse(ESP_ADC_CAL_VAL_EFUSE_TP) == ESP_OK) {
        ESP_LOGI(TAG, "eFuse Two Point: รองรับ");
    } else {
        ESP_LOGI(TAG, "eFuse Two Point: ไม่รองรับ");
    }
    // ตรวจสอบว่า Vref ถูกเขียนลงใน eFuse หรือไม่
    if (esp_adc_cal_check_efuse(ESP_ADC_CAL_VAL_EFUSE_VREF) == ESP_OK) {
        ESP_LOGI(TAG, "eFuse Vref: รองรับ");
    } else {
        ESP_LOGI(TAG, "eFuse Vref: ไม่รองรับ");
    }
    return true;
}

static void print_char_val_type(esp_adc_cal_value_t val_type)
{
    if (val_type == ESP_ADC_CAL_VAL_EFUSE_TP) {
        ESP_LOGI(TAG, "ใช้การปรับเทียบแบบ Two Point Value");
    } else if (val_type == ESP_ADC_CAL_VAL_EFUSE_VREF) {
        ESP_LOGI(TAG, "ใช้การปรับเทียบแบบ eFuse Vref");
    } else {
        ESP_LOGI(TAG, "ใช้การปรับเทียบแบบ Default Vref");
    }
}

void app_main(void)
{
    // ตรวจสอบว่า Two Point หรือ Vref ถูกเขียนลงใน eFuse หรือไม่
    check_efuse();

    // กำหนดค่า ADC
    adc1_config_width(ADC_WIDTH_BIT_12);
    adc1_config_channel_atten(POTENTIOMETER_CHANNEL, ADC_ATTEN_DB_11);

    // ปรับเทียบ ADC
    adc_chars = calloc(1, sizeof(esp_adc_cal_characteristics_t));
    esp_adc_cal_value_t val_type = esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_12, DEFAULT_VREF, adc_chars);
    print_char_val_type(val_type);

    ESP_LOGI(TAG, "ทดสอบ ADC Potentiometer ของ ESP32");
    ESP_LOGI(TAG, "Pin: GPIO34 (ADC1_CH6)");
    ESP_LOGI(TAG, "ช่วง: 0-3.3V");
    ESP_LOGI(TAG, "ความละเอียด: 12-bit (0-4095)");
    ESP_LOGI(TAG, "Attenuation: 11dB");
    ESP_LOGI(TAG, "-------------------------");

    // อ่านค่า ADC1 อย่างต่อเนื่อง
    while (1) {
        uint32_t adc_reading = 0;
        
        // การสุ่มสัญญาณหลายครั้ง
        for (int i = 0; i < NO_OF_SAMPLES; i++) {
            adc_reading += adc1_get_raw((adc1_channel_t)POTENTIOMETER_CHANNEL);
        }
        adc_reading /= NO_OF_SAMPLES;
        
        // แปลง adc_reading เป็นแรงดันในหน่วย mV
        uint32_t voltage_mv = esp_adc_cal_raw_to_voltage(adc_reading, adc_chars);
        float voltage = voltage_mv / 1000.0;
        
        // แปลงเป็นเปอร์เซ็นต์
        float percentage = (adc_reading / 4095.0) * 100.0;
        
        // แสดงผลลัพธ์
        ESP_LOGI(TAG, "ค่า ADC: %d | แรงดัน: %.2fV | เปอร์เซ็นต์: %.1f%%", 
                 adc_reading, voltage, percentage);
        
        vTaskDelay(pdMS_TO_TICKS(500));  // หน่วงเวลา 500ms
    }
}

### คำถามสำหรับการทดลองที่ 1
1. เมื่อหมุน Potentiometer ไปทางซ้ายสุด ค่า ADC ที่ได้คือเท่าไร?

ตอบ ADC_POT: ค่า ADC: 0 | แรงดัน: 0.14V | เปอร์เซ็นต์: 0.0%

2. เมื่อหมุน Potentiometer ไปทางขวาสุด ค่า ADC ที่ได้คือเท่าไร?

ตอบ ADC_POT: ค่า ADC: 4095 | แรงดัน: 3.16V | เปอร์เซ็นต์: 100.0%

3. หากต้องการให้ค่า ADC อยู่ประมาณ 2048 ต้องหมุน Potentiometer ไปที่ตำแหน่งใด?

ตอบ ADC_POT: ค่า ADC: 2048 | แรงดัน: 1.84V | เปอร์เซ็นต์: 50.0%

4. ความผิดพลาดของการวัด (Error) มีสาเหตุมาจากอะไรบ้าง?

ตอบ - การวัด ADC บนไมโครคอนโทรลเลอร์มีความไวต่อ noise จากสาย USB, switching power supply, และวงจรใกล้เคียง

- แม้ ESP32 จะมี eFuse Vref แต่ยังมีความคลาดเคลื่อน ทำให้ค่าแรงดันที่แปลงไม่ตรง 100%

- แรงดัน 3.3V อาจไม่คงที่ตลอดเวลา ทำให้ค่า ADC ผันผวน และอุณหภูมิก็มีผลต่อความเสถียรของ ADC

### การบันทึกผล
สร้างตารางบันทึกผลการทดลอง:



| ตำแหน่ง Potentiometer | ค่า ADC | แรงดัน (V) | เปอร์เซ็นต์ (%) |
|----------------------|---------|-------------|----------------|
| ซ้ายสุด               |   	4095      |   3.15          |       100         |
| 1/4                  |     1046    |      1       |      25.5          |
| 1/2 (กลาง)            |      	2031   |    1.81         |      49.6          |
| 3/4                  |     3089    |       2.64      |       75.4         |
| ขวาสุด                |     	0    |       0.14      |        0        |

## การทดลองที่ 2 - การอ่านค่าเซนเซอร์แสง (LDR)

### วัตถุประสงค์
เพื่อศึกษาการใช้ ADC อ่านค่าจากเซนเซอร์ที่มีการเปลี่ยนแปลงตามสิ่งแวดล้อม

### วงจร
เชื่อมต่อ LDR กับ ESP32 ดังนี้


![](./Images/ESP32-LDR.png)


### หลักการทำงาน
- LDR เป็นตัวต้านทานที่เปลี่ยนค่าตามแสง
- เมื่อแสงมาก ความต้านทาน LDR ลดลง → แรงดันที่ ADC อ่านได้เพิ่มขึ้น
- เมื่อแสงน้อย ความต้านทาน LDR เพิ่มขึ้น → แรงดันที่ ADC อ่านได้ลดลง

In [ ]:
// การทดลองที่ 2: เซนเซอร์แสง LDR
// Light Sensor with ESP32 ADC using ESP-IDF

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "freertos/FreeRTOS.h"
#include "freertos/task.h"
#include "driver/adc.h"
#include "esp_adc_cal.h"
#include "esp_log.h"

// กำหนด pin ที่ใช้
#define LDR_CHANNEL ADC1_CHANNEL_7  // GPIO35 (ADC1_CH7)
#define DEFAULT_VREF    1100        // ใช้ adc2_vref_to_gpio() เพื่อให้ได้ค่าประมาณที่ดีกว่า
#define NO_OF_SAMPLES   64          // การสุ่มสัญญาณหลายครั้ง

static const char *TAG = "ADC_LDR";
static esp_adc_cal_characteristics_t *adc_chars;

static bool check_efuse(void)
{
    // ตรวจสอบว่า TP ถูกเขียนลงใน eFuse หรือไม่
    if (esp_adc_cal_check_efuse(ESP_ADC_CAL_VAL_EFUSE_TP) == ESP_OK) {
        ESP_LOGI(TAG, "eFuse Two Point: รองรับ");
    } else {
        ESP_LOGI(TAG, "eFuse Two Point: ไม่รองรับ");
    }
    // ตรวจสอบว่า Vref ถูกเขียนลงใน eFuse หรือไม่
    if (esp_adc_cal_check_efuse(ESP_ADC_CAL_VAL_EFUSE_VREF) == ESP_OK) {
        ESP_LOGI(TAG, "eFuse Vref: รองรับ");
    } else {
        ESP_LOGI(TAG, "eFuse Vref: ไม่รองรับ");
    }
    return true;
}

static void print_char_val_type(esp_adc_cal_value_t val_type)
{
    if (val_type == ESP_ADC_CAL_VAL_EFUSE_TP) {
        ESP_LOGI(TAG, "ใช้การปรับเทียบแบบ Two Point Value");
    } else if (val_type == ESP_ADC_CAL_VAL_EFUSE_VREF) {
        ESP_LOGI(TAG, "ใช้การปรับเทียบแบบ eFuse Vref");
    } else {
        ESP_LOGI(TAG, "ใช้การปรับเทียบแบบ Default Vref");
    }
}

void app_main(void)
{
    // ตรวจสอบว่า Two Point หรือ Vref ถูกเขียนลงใน eFuse หรือไม่
    check_efuse();

    // กำหนดค่า ADC
    adc1_config_width(ADC_WIDTH_BIT_12);
    adc1_config_channel_atten(LDR_CHANNEL, ADC_ATTEN_DB_11);

    // ปรับเทียบ ADC
    adc_chars = calloc(1, sizeof(esp_adc_cal_characteristics_t));
    esp_adc_cal_value_t val_type = esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_11, ADC_WIDTH_BIT_12, DEFAULT_VREF, adc_chars);
    print_char_val_type(val_type);

    ESP_LOGI(TAG, "ทดสอบเซนเซอร์แสง LDR ของ ESP32");
    ESP_LOGI(TAG, "Pin: GPIO35 (ADC1_CH7)");
    ESP_LOGI(TAG, "ช่วง: 0-3.3V");
    ESP_LOGI(TAG, "ความละเอียด: 12-bit (0-4095)");
    ESP_LOGI(TAG, "Attenuation: 11dB");
    ESP_LOGI(TAG, "------------------------");

    // อ่านค่า ADC จาก LDR อย่างต่อเนื่อง
    while (1) {
        uint32_t adc_reading = 0;
        
        // การสุ่มสัญญาณหลายครั้ง
        for (int i = 0; i < NO_OF_SAMPLES; i++) {
            adc_reading += adc1_get_raw((adc1_channel_t)LDR_CHANNEL);
        }
        adc_reading /= NO_OF_SAMPLES;
        
        // แปลง adc_reading เป็นแรงดันในหน่วย mV
        uint32_t voltage_mv = esp_adc_cal_raw_to_voltage(adc_reading, adc_chars);
        float voltage = voltage_mv / 1000.0;
        
        // แปลงเป็นระดับแสง (สมมติ)
        float lightLevel = (adc_reading / 4095.0) * 100.0;
        
        // กำหนดสถานะแสง
        const char* lightStatus;
        if (lightLevel < 20) {
            lightStatus = "มืด";
        } else if (lightLevel < 50) {
            lightStatus = "แสงน้อย";
        } else if (lightLevel < 80) {
            lightStatus = "แสงปานกลาง";
        } else {
            lightStatus = "แสงจ้า";
        }
        
        // แสดงผลลัพธ์
        ESP_LOGI(TAG, "ADC: %d | แรงดัน: %.2fV | ระดับแสง: %.1f%% | สถานะ: %s", 
                 adc_reading, voltage, lightLevel, lightStatus);
        
        vTaskDelay(pdMS_TO_TICKS(1000));  // หน่วงเวลา 1 วินาที
    }
}

## การทดลองที่ 3 - การปรับปรุงความแม่นยำของ ADC

### วัตถุประสงค์
เพื่อศึกษาวิธีการปรับปรุงความแม่นยำของการอ่านค่า ADC ด้วยเทคนิคต่างๆ

### เทคนิคที่ใช้
1. **Oversampling**: การอ่านค่าหลายครั้งแล้วหาค่าเฉลี่ย
2. **Calibration**: การปรับค่า offset และ gain
3. **Filtering**: การกรองสัญญาณรบกวน

In [ ]:
﻿/*
 * LDR + Buzzer + LED (ภาษาไทย) + ปุ่มกด:
 * - แสงน้อย -> หรี่ LED, แสงมาก -> เพิ่มความสว่าง
 * - ต่ำกว่าเกณฑ์ -> Buzzer ดัง (มีฮิสเทอรีซีส)
 * - ปุ่ม BOOT (GPIO0):
 *      กดสั้น  <1s  -> ปิดเสียงเตือนชั่วคราว 30s (Mute)
 *      กดค้าง >=1s  -> คาลิเบรตเกณฑ์จากสภาพแสงปัจจุบัน (TH_ON/TH_OFF)
 * Log: "ADC: <raw> | แรงดัน: <V>V | ระดับแสง: <เปอร์เซ็นต์>% | สถานะ: <ข้อความ>"
 *
 * วงจรเซนเซอร์ตามเดิม:
 *   3V3 -> LDR -> โหนดสัญญาณ -> (R=470Ω->GND, C=1nF->GND) -> GPIO35 (ADC1_CH7)
 * Buzzer (Active) + Q1 2N3904: 3V3 -> (+)Buzzer, (-)Buzzer -> C Q1, E Q1->GND, B Q1<-R10k<-GPIO18
 * LED สถานะ: GPIO2 -> LED -> R 220~470Ω -> GND
 */

#include <stdio.h>
#include <string.h>
#include <stdbool.h>
#include "freertos/FreeRTOS.h"
#include "freertos/task.h"
#include "driver/gpio.h"
#include "driver/ledc.h"
#include "driver/adc.h"          // legacy ADC (IDF 5.x ยังใช้ได้)
#include "esp_adc_cal.h"         // legacy calibration
#include "esp_log.h"
#include "esp_err.h"
#include "esp_rom_sys.h"         // esp_rom_delay_us()

#define TAG "LDR_BTN_BUZZ_LED_TH"

/* --------- พินใช้งาน --------- */
#define LDR_ADC_CHANNEL   ADC1_CHANNEL_7     // GPIO35
#define LDR_ADC_GPIO      35
#define BUZZER_GPIO       GPIO_NUM_18        // ผ่าน Q1 2N3904 (Active-high)
#define LED_GPIO          GPIO_NUM_2         // LED สถานะ
#define BTN_GPIO          GPIO_NUM_0         // ปุ่ม BOOT บนบอร์ด (pull-up, กด=0)

/* --------- พารามิเตอร์อ่านค่า/กรอง --------- */
#define DEFAULT_VREF      1100               // mV
#define OVERSAMPLES       64                 // จำนวนครั้งอ่าน/รอบ
#define FILTER_SIZE       10                 // Moving Average
#define SAMPLE_PERIOD_MS  100                // คาบการอ่าน/รายงาน (ทำให้ปุ่มตอบสนองไวขึ้น)

/* --------- PWM (LEDC) สำหรับ LED --------- */
#define LEDC_TIMER_IDX    LEDC_TIMER_0
#define LEDC_MODE_SEL     LEDC_LOW_SPEED_MODE
#define LEDC_CHANNEL_IDX  LEDC_CHANNEL_0
#define LEDC_DUTY_RES     LEDC_TIMER_13_BIT  // 13-bit (0..8191)
#define LEDC_FREQ_HZ      5000

/* --------- เกณฑ์แจ้งเตือน (ค่าเริ่มต้น) --------- */
#define THRESHOLD_MV_ON_DEFAULT   1000       // < เกณฑ์นี้ = มืด -> เปิด Buzzer
#define THRESHOLD_MV_OFF_DEFAULT  1200       // > เกณฑ์นี้ = หายมืด -> ปิด Buzzer

/* --------- สเกลแรงดันสำหรับคิดเปอร์เซ็นต์แสง --------- */
#define MV_MIN_SCALE      50                 // ~มืดสุด
#define MV_MAX_SCALE      3300               // ~สว่างสุด (3.3V)

/* --------- ปุ่มกด --------- */
#define BTN_DEBOUNCE_MS   30
#define BTN_LONG_MS       1000
#define MUTE_MS           30000              // กดสั้น -> ปิดเสียง 30 วินาที

/* --------- ตัวแปรภายใน --------- */
static esp_adc_cal_characteristics_t *adc_chars;
static bool  filt_init  = false;
static bool  buzzer_on  = false;

/* เกณฑ์ runtime (เปลี่ยนได้ด้วยการกดปุ่มค้าง) */
static int   th_on_mv  = THRESHOLD_MV_ON_DEFAULT;
static int   th_off_mv = THRESHOLD_MV_OFF_DEFAULT;

/* mute จนถึงเวลา tick นี้ */
static TickType_t mute_until_tick = 0;

/* --------- ฟังก์ชันช่วย --------- */
static void ledc_init(void) {
    ledc_timer_config_t tcfg = {
        .speed_mode       = LEDC_MODE_SEL,
        .timer_num        = LEDC_TIMER_IDX,
        .duty_resolution  = LEDC_DUTY_RES,
        .freq_hz          = LEDC_FREQ_HZ,
        .clk_cfg          = LEDC_AUTO_CLK
    };
    ESP_ERROR_CHECK(ledc_timer_config(&tcfg));

    ledc_channel_config_t cconfig = {
        .gpio_num       = LED_GPIO,
        .speed_mode     = LEDC_MODE_SEL,
        .channel        = LEDC_CHANNEL_IDX,
        .intr_type      = LEDC_INTR_DISABLE,
        .timer_sel      = LEDC_TIMER_IDX,
        .duty           = 0,
        .hpoint         = 0
    };
    ESP_ERROR_CHECK(ledc_channel_config(&cconfig));
}

static inline void set_led_duty(uint32_t duty) {
    ESP_ERROR_CHECK(ledc_set_duty(LEDC_MODE_SEL, LEDC_CHANNEL_IDX, duty));
    ESP_ERROR_CHECK(ledc_update_duty(LEDC_MODE_SEL, LEDC_CHANNEL_IDX));
}

static uint32_t oversample_read(adc1_channel_t ch, int samples) {
    uint64_t sum = 0;
    for (int i = 0; i < samples; ++i) {
        sum += adc1_get_raw(ch);
        esp_rom_delay_us(100);
    }
    return (uint32_t)(sum / samples);
}

static float moving_average(float x) {
    static float buf[FILTER_SIZE];
    static int idx = 0, count = 0;
    if (!filt_init) {
        for (int i = 0; i < FILTER_SIZE; ++i) buf[i] = x;
        idx = 0; count = FILTER_SIZE; filt_init = true;
        return x;
    }
    buf[idx] = x;
    idx = (idx + 1) % FILTER_SIZE;
    float s = 0.0f;
    for (int i = 0; i < count; ++i) s += buf[i];
    return s / count;
}

static float clampf(float v, float lo, float hi) {
    if (v < lo) return lo;
    if (v > hi) return hi;
    return v;
}

/* ปุ่ม: คืนค่าเหตุการณ์ (ไม่มี/กดสั้น/กดค้าง) */
typedef enum { BTN_EVT_NONE=0, BTN_EVT_SHORT, BTN_EVT_LONG } btn_evt_t;

static btn_evt_t poll_button(void) {
    static int stable = 1;  // pull-up -> ว่าง = 1
    static TickType_t last_change = 0;
    static TickType_t press_tick  = 0;

    TickType_t now = xTaskGetTickCount();
    int level = gpio_get_level(BTN_GPIO);

    if (level != stable) {
        // debounce
        if ((now - last_change) * portTICK_PERIOD_MS >= BTN_DEBOUNCE_MS) {
            last_change = now;
            stable = level;
            if (stable == 0) {
                // กดลง
                press_tick = now;
            } else {
                // ปล่อยปุ่ม -> คำนวณเวลาที่กด
                TickType_t held_ms = (now - press_tick) * portTICK_PERIOD_MS;
                if (held_ms >= BTN_LONG_MS) return BTN_EVT_LONG;
                if (held_ms >= BTN_DEBOUNCE_MS) return BTN_EVT_SHORT;
            }
        }
    }
    return BTN_EVT_NONE;
}

/* คาลิเบรตเกณฑ์จากสภาพแสงปัจจุบัน */
static void recalibrate_thresholds(adc1_channel_t ch) {
    const int N = 20;
    uint64_t sum_mv = 0;
    for (int i = 0; i < N; ++i) {
        uint32_t raw = oversample_read(ch, OVERSAMPLES);
        uint32_t mv  = esp_adc_cal_raw_to_voltage(raw, adc_chars);
        sum_mv += mv;
        vTaskDelay(pdMS_TO_TICKS(20));
    }
    int base = (int)(sum_mv / N);
    // ตั้งฮิสเทอรีซีสรอบ ๆ ค่าฐาน
    th_on_mv  = base - 200;  // ต่ำกว่านี้ถือว่ามืดมาก (เปิด buzzer)
    th_off_mv = base + 100;  // สูงกว่านี้ถือว่ากลับสว่าง (ปิด buzzer)

    if (th_on_mv  < MV_MIN_SCALE) th_on_mv  = MV_MIN_SCALE;
    if (th_off_mv > MV_MAX_SCALE) th_off_mv = MV_MAX_SCALE;
    if (th_off_mv <= th_on_mv + 50) th_off_mv = th_on_mv + 50; // กันชน

    ESP_LOGW(TAG, "คาลิเบรตใหม่จากสภาพแสง: base=%dmV -> TH_ON=%dmV, TH_OFF=%dmV",
             base, th_on_mv, th_off_mv);
}

/* --------- main --------- */
void app_main(void) {
    ESP_LOGI(TAG,
        "เริ่มระบบ | LDR: GPIO%d(ADC1_CH7) | BUZZER: GPIO%d | LED: GPIO%d | BTN: GPIO%d | OVERSAMPLES=%d FILTER_SIZE=%d",
        LDR_ADC_GPIO, BUZZER_GPIO, LED_GPIO, BTN_GPIO, OVERSAMPLES, FILTER_SIZE);

    /* ตั้ง BUZZER เป็นเอาต์พุต */
    gpio_config_t buz = {
        .pin_bit_mask = 1ULL << BUZZER_GPIO,
        .mode = GPIO_MODE_OUTPUT,
        .pull_up_en = 0, .pull_down_en = 0,
        .intr_type = GPIO_INTR_DISABLE
    };
    ESP_ERROR_CHECK(gpio_config(&buz));
    gpio_set_level(BUZZER_GPIO, 0);

    /* ตั้ง LED PWM */
    ledc_init();

    /* ตั้งปุ่ม (pull-up, กด=0) */
    gpio_config_t btn = {
        .pin_bit_mask = 1ULL << BTN_GPIO,
        .mode = GPIO_MODE_INPUT,
        .pull_up_en = 1, .pull_down_en = 0,
        .intr_type = GPIO_INTR_DISABLE
    };
    ESP_ERROR_CHECK(gpio_config(&btn));

    /* ตั้ง ADC legacy + calibration (ใช้ DB_12 แทน DB_11) */
    adc1_config_width(ADC_WIDTH_BIT_12);
    adc1_config_channel_atten(LDR_ADC_CHANNEL, ADC_ATTEN_DB_12); // ~0..3.3V
    adc_chars = calloc(1, sizeof(esp_adc_cal_characteristics_t));
    (void)esp_adc_cal_characterize(ADC_UNIT_1, ADC_ATTEN_DB_12, ADC_WIDTH_BIT_12, DEFAULT_VREF, adc_chars);

    const uint32_t duty_max = (1U << LEDC_DUTY_RES) - 1U;

    while (1) {
        /* --- อ่านเหตุการณ์ปุ่ม --- */
        btn_evt_t evt = poll_button();
        TickType_t now = xTaskGetTickCount();
        if (evt == BTN_EVT_SHORT) {
            mute_until_tick = now + pdMS_TO_TICKS(MUTE_MS);
            ESP_LOGW(TAG, "ปุ่ม: กดสั้น -> ปิดเสียงเตือนชั่วคราว %d ms", (int)MUTE_MS);
        } else if (evt == BTN_EVT_LONG) {
            recalibrate_thresholds(LDR_ADC_CHANNEL);
        }

        /* 1) อ่านค่า + ทำให้เรียบ */
        uint32_t raw_os = oversample_read(LDR_ADC_CHANNEL, OVERSAMPLES);
        float raw_ma = moving_average((float)raw_os);
        uint32_t raw = (uint32_t)raw_ma;

        /* 2) แปลงเป็น mV และเปอร์เซ็นต์แสง */
        uint32_t mv = esp_adc_cal_raw_to_voltage(raw, adc_chars);
        float mv_c = clampf((float)mv, MV_MIN_SCALE, MV_MAX_SCALE);
        float light_pct = 100.0f * (mv_c - MV_MIN_SCALE) / (float)(MV_MAX_SCALE - MV_MIN_SCALE);
        if (light_pct < 0) light_pct = 0; else if (light_pct > 100) light_pct = 100;

        /* 3) LED สว่างตามแสง (สู้แสง): แสงมาก -> LED มาก */
        uint32_t duty = (uint32_t)((light_pct / 100.0f) * duty_max);
        set_led_duty(duty);

        /* 4) เปิด/ปิด Buzzer ด้วยฮิสเทอรีซีส + mute */
        bool mute_active = (now < mute_until_tick);
        if (!buzzer_on && mv < th_on_mv) {
            buzzer_on = true;
        } else if (buzzer_on && mv > th_off_mv) {
            buzzer_on = false;
        }
        gpio_set_level(BUZZER_GPIO, (buzzer_on && !mute_active) ? 1 : 0);

        /* 5) รายงานภาษาไทยสั้น ๆ */
        const char* status;
        if (mute_active) {
            status = "ปิดเสียงชั่วคราว (Mute)";
        } else if (buzzer_on) {
            status = "แสงน้อยกว่าเกณฑ์ -> Buzzer ดัง";
        } else if (mv < th_off_mv && mv >= th_on_mv) {
            status = "เข้าใกล้เกณฑ์";
        } else {
            status = "แสงปกติ";
        }

        ESP_LOGI(TAG, "ADC: %u | แรงดัน: %.2fV | ระดับแสง: %.1f%% | สถานะ: %s (TH_ON=%dmV, TH_OFF=%dmV)",
                 raw, mv / 1000.0f, light_pct, status, th_on_mv, th_off_mv);

        vTaskDelay(pdMS_TO_TICKS(SAMPLE_PERIOD_MS));
    }
}

## โจทย์ท้าทาย 

1. ออกแบบระบบควบคุมแสง LED โดยใช้ค่าจาก LDR


### วงจรที่ใช้

![](./Images/ESP32-LDR-LED.png) 

### การทำงาน 
- เมื่อแสงน้อยให้หรี่ LED เพื่อไม่ให้แสงจ้าเกินไป
- เมื่อแสงมากให้เพิ่มความสว่างของ LED เพื่อให้สู้แสงภายนอกได้

2. สร้างระบบเตือนเมื่อค่าเซนเซอร์เกินขีดจำกัด

### วงจรที่ใช้

![](./Images/ESP32-LDR-BUZZER.png)

###  การทำงาน 
- เมื่อแสงน้อยกว่าเกณฑ์ที่ตั้งไว้ (เช่น ค่าที่อ่านได้น้อยกว่า 1000) ให้ Buzzer ดังเตือน

### เพิ่มเติม 
- อาจจะเพิ่ม LED แล้วสั่งให้ติดสว่างขึ้นเพื่อแสดงสถานะว่า LDR ยังคงทำงานอยู่

## การวิเคราะห์ผลและแบบฝึกหัด

### คำถามท้าทาย
การคำนวณความละเอียด:

ADC 12-bit มีความละเอียดเท่าไร? (ในหน่วย mV เมื่อใช้ช่วง 0-3.3V)
ตอบ ถ้าใช้สูตร 3300/4096 ประมาณ 0.8057 mV
หากต้องการความละเอียด 1mV ต้องใช้ ADC กี่บิต?
ตอบ LSB = 3300/(2^N - 1) mV <= 1 -> N >= log2(3301) ~ 11.7 -> ปัดขึ้นเป็น 12 บิต
การวิเคราะห์ error:

Quantization error สูงสุดของ 12-bit ADC คือเท่าไร?
ตอบ error สูงสุด ~ +/-0.403 mV
เหตุใดการใช้ oversampling จึงช่วยลด noise ได้?
ตอบ ค่าเฉลี่ยของหลายๆ ตัวอย่างจะถัวเฉลี่ย noise แบบสุ่มให้เล็กลง เฉลี่ย M ตัวอย่าง -> ส่วนเบี่ยงเบนของ noise ลดลงประมาณ 1/sqrt(M)

